<h1><span style="color:red">Add Color Statistics for Images in Image Collection</span></h1>

### This sample notebook will read survey images and add lightness, hue, brightness, saturation, and RGB values to a new survey version

## 1. Retrieve survey parameters from the URL

In [ ]:
%%javascript
function getQueryStringValue (key)
{  
    return unescape(window.location.search.replace(new RegExp("^(?:.*[&\\?]" + escape(key).replace(/[\.\+\*]/g, "\\$&") + "(?:\\=([^&]*))?)?.*$", "i"), "$1"));
}
IPython.notebook.kernel.execute("survey_url='".concat(getQueryStringValue("surveyurl")).concat("'"));
IPython.notebook.kernel.execute("views='".concat(getQueryStringValue("views")).concat("'"));
IPython.notebook.kernel.execute("view='".concat(getQueryStringValue("view")).concat("'"));
IPython.notebook.kernel.execute("user='".concat(getQueryStringValue("user")).concat("'"));
IPython.notebook.kernel.execute("csv_file='".concat(getQueryStringValue("csv")).concat("'")); 
IPython.notebook.kernel.execute("dzc_file='".concat(getQueryStringValue("dzc")).concat("'")); 
IPython.notebook.kernel.execute("params='".concat(getQueryStringValue("params")).concat("'")); 
IPython.notebook.kernel.execute("active_object='".concat(getQueryStringValue("activeobject")).concat("'")); 
IPython.notebook.kernel.execute("full_notebook_url='" + window.location + "'"); 

In [ ]:
# common imports
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import Markdown, display

import pandas as pd
pd.set_option('display.max_colwidth', 0)
    
import numpy as np
import panel as pn

pn.extension()
def printmd(string):
    display(Markdown(string))

absolutePath = "../../temp_csvs/"

# local imports
import sys
sys.path.insert(1, '../../helpers')
import panel_libs as panellibs
import suave_integration as suaveint

# specific imports
from PIL import Image, ImageStat
import glob, os
import csv
import requests
import re



## 2. Read the survey file and navigate to full-size images

In [ ]:
# read the csv file
df = panellibs.extract_data(absolutePath + csv_file)
dflen = len(df.columns)

localdzc = dzc_file.replace("https://maxim.ucsd.edu/dzgen/lib-staging-uploads","/lib-nfs/dzgen")
full_images_location = localdzc.replace("/content.dzc","/full_images/")

## 3. Specify color characteristics to extract from images

In [ ]:
a = widgets.Dropdown(options=["Compute RGB values", "Don't compute RGB values"])
b = widgets.Dropdown(options=["Compute Lightness values", "Don't compute Lightness values"])
c = widgets.Dropdown(options=["Omit Root-Mean-Square for the selected bands", "Include Root-Mean-Square for the selected bands"])
ui = widgets.VBox([a, b, c])
def f(a, b, c):
    return ((a, b, c))

formula = widgets.interactive_output(f, {'a': a, 'b': b, 'c': c})

display(ui, formula)

printmd("<b><span style='color:red'>Select variables to include in the output, then run the next cell</span></b>")


In [ ]:
# include RGB values
printmd("<b><span style='color:red'>The following color characteristics will be added:</span></b>")
RGB = Light = rms = False
if (a.value == 'Compute RGB values'):
    print('Red-Green-Blue bands: Mean, Median, Standard Deviation: YES')
    RGB = True
if (b.value == 'Compute Lightness values'):
    print('Lightness: Mean, Median, Standard Deviation: YES')
    Light = True
if (c.value == 'Include Root-Mean-Square for the selected bands'):
    print('Root-Mean-Square for the above bands: YES')
    rms = True
    


## 4. Specify band statistics functions ##

In [ ]:
def lightness( im_file ):
    im = Image.open(im_file).convert('L')
    stat = ImageStat.Stat(im)
    return [stat.mean[0], stat.median[0], stat.rms[0], stat.stddev[0]]

def RGBstats ( im_file ):
    im = Image.open(im_file).convert('RGB')
    stat = ImageStat.Stat(im)
    return [
        [stat.mean[0], stat.median[0], stat.rms[0], stat.stddev[0]],
        [stat.mean[1], stat.median[1], stat.rms[1], stat.stddev[1]],
        [stat.mean[2], stat.median[2], stat.rms[2], stat.stddev[2]]
    ]
def HSVstats ( im_file ):
    im = Image.open(im_file).convert('HSV')
    stat = ImageStat.Stat(im)
    return [
        [stat.mean[0], stat.median[0], stat.rms[0], stat.stddev[0]],
        [stat.mean[1], stat.median[1], stat.rms[1], stat.stddev[1]],
        [stat.mean[2], stat.median[2], stat.rms[2], stat.stddev[2]]
    ]

## 5. Process images

In [ ]:
# Processing image file and adding to data frame
all_data = []
counter = 0
a = widgets.Label(value="0% done")
display(a)

# Uncomment only if want to process based on level 8 tiles. It is faster but not as accurate.
# for dzi in glob.glob("*.dzi"):
#     file = os.path.join(os.getcwd(), dzi[:-4] +"_files", "8", "0_0.jpg")

# numfiles = len(glob.glob("*.png"))
numfiles = len(glob.glob(full_images_location+"*.png"))

for file in glob.glob(full_images_location+"*.png"):
    file_data = {} # data for this particular file
    try:
        l1 = lightness(file)
        h1 = HSVstats(file)
        r1 = RGBstats(file)
        file_data['#img'] = os.path.basename(file)[:-4]

        file_data['Lightness_mean']   = l1[0]
        file_data['Lightness_median'] = l1[1]
        file_data['Lightness_rms']    = l1[2]
        file_data['Lightness_std']    = l1[3]

        file_data['Hue_mean']          = h1[0][0]
        file_data['Hue_median']        = h1[0][1]
        file_data['Hue_rms']           = h1[0][2]
        file_data['Hue_std']           = h1[0][3]
        file_data['Saturation_mean']   = h1[1][0]
        file_data['Saturation_median'] = h1[1][1]
        file_data['Saturation_rms']    = h1[1][2]
        file_data['Saturation_std']    = h1[1][3]
        file_data['Brightness_mean']        = h1[2][0]
        file_data['Brightness_median']      = h1[2][1]
        file_data['Brightness_rms']         = h1[2][2]
        file_data['Brightness_std']         = h1[2][3]
        
        file_data['Red_mean']          = r1[0][0]
        file_data['Red_median']        = r1[0][1]
        file_data['Red_rms']           = r1[0][2]
        file_data['Red_std']           = r1[0][3]
        file_data['Green_mean']        = r1[1][0]
        file_data['Green_median']      = r1[1][1]
        file_data['Green_rms']         = r1[1][2]
        file_data['Green_std']         = r1[1][3]
        file_data['Blue_mean']         = r1[2][0]
        file_data['Blue_median']       = r1[2][1]
        file_data['Blue_rms']          = r1[2][2]
        file_data['Blue_std']          = r1[2][3]
        
        
        all_data.append(file_data)    
    except Exception as e:
        print(file, "There was an issue: ", e)
    counter += 1
    a.value = str(int(counter / numfiles * 100)) + "% done"
        
newdf = pd.DataFrame(all_data).fillna('')
printmd("<b><span style='color:red'>All files processed</span></b>")

## 6. Add the result to the survey dataframe

In [ ]:
columnTitles = [
    '#img',
    'Brightness_mean','Brightness_median']
if rms:
    columnTitles.append('Brightness_rms')
columnTitles.extend(('Brightness_std','Hue_mean','Hue_median'))
if rms:
    columnTitles.append('Hue_rms')
columnTitles.extend(('Hue_std','Saturation_mean','Saturation_median'))
if rms:
    columnTitles.append('Saturation_rms')
columnTitles.append('Saturation_std')

if Light:
    columnTitles.extend(('Lightness_mean', 'Lightness_median'))
    if rms: 
        columnTitles.append('Lightness_rms')
    columnTitles.append('Lightness_std')
    
if RGB:
    columnTitles.extend(('Red_mean','Red_median'))
    if rms:
        columnTitles.append('Red_rms')
    columnTitles.extend(('Red_std','Green_mean','Green_median'))
    if rms:
        columnTitles.append('Green_rms')
    columnTitles.extend(('Green_std','Blue_mean','Blue_median'))
    if rms:
        columnTitles.append('Blue_rms')
    columnTitles.append('Blue_std')

printmd("<br><b><span style='color:red'>The following columns will be added:</span></b>")    
print(columnTitles[1:])

newdf = newdf[columnTitles]


In [ ]:
# add #number to numeric column names

collist = []
for col in newdf.columns:
    col1 = col+"#number"
    collist.append(col1)
newdf.columns = collist
newdf = newdf.rename(columns={'#img#number': '#img'})
df = pd.merge(df, newdf, on='#img', how='outer')
printmd("<b><span style='color:red'>Dataframe created</span></b>")

## 7. Examine the generated dataframe

In [ ]:
panellibs.slider(newdf)

## 8. Save the new version of CSV file, and give a name to new survey

In [ ]:
new_file = suaveint.save_csv_file(df, absolutePath, csv_file)

In [ ]:
#Input survey name

from IPython.display import display
input_text = widgets.Text(placeholder='Enter Survey Name...')
output_text = widgets.Text()

def bind_input_to_output(sender):
    output_text.value = input_text.value

# Tell the text input widget to call bind_input_to_output() on submit
input_text.on_submit(bind_input_to_output)

printmd("<b><span style='color:red'>Input survey name here, press Enter, and then run the next cell:</span></b>")
# Display input text box widget for input
display(input_text)

display(output_text)


In [ ]:
#Print survey name
survey_name = output_text.value
printmd("<b><span style='color:red'>Survey Name is: </span></b>" + survey_name)

## 9. Generate the survey and create survey URL

In [ ]:
suaveint.create_survey(survey_url,new_file, survey_name, dzc_file, user, csv_file, view, views)